<a href="https://colab.research.google.com/github/addicted-ai/Housing-Price-Prediction/blob/main/Housing-Price-Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Necessary Libraries

In [4]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [6]:
# For train-test split
from sklearn.model_selection import train_test_split

# For Min-Max Scaler
from sklearn.preprocessing import MinMaxScaler

# for automated feature selection using RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# StatsModel for model Building as it gives better Statistics
import statsmodels.api as sm

# For Model Evaluation
from sklearn.metrics import r2_score

## Reading Dataset

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/addicted-ai/Housing-Price-Prediction/main/melb_data.csv')
df.head(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
5,Abbotsford,129 Charles St,2,h,941000.0,S,Jellis,7/05/2016,2.5,3067.0,2.0,1.0,0.0,181.0,NaN,NaN,Yarra,-37.8041,144.9953,Northern Metropolitan,4019.0
6,Abbotsford,124 Yarra St,3,h,1876000.0,S,Nelson,7/05/2016,2.5,3067.0,4.0,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,Abbotsford,98 Charles St,2,h,1636000.0,S,Nelson,8/10/2016,2.5,3067.0,2.0,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0
8,Abbotsford,6/241 Nicholson St,1,u,300000.0,S,Biggin,8/10/2016,2.5,3067.0,1.0,1.0,1.0,0.0,NaN,NaN,Yarra,-37.8008,144.9973,Northern Metropolitan,4019.0
9,Abbotsford,10 Valiant St,2,h,1097000.0,S,Biggin,8/10/2016,2.5,3067.0,3.0,1.0,2.0,220.0,75.0,1900.0,Yarra,-37.8010,144.9989,Northern Metropolitan,4019.0


In [16]:
df.describe(percentiles=[0.1,0.25,0.5,0.75,0.9,0.95,0.97,0.99,1])

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
10%,2.000000,4.800000e+05,3.500000,3020.000000,2.000000,1.000000,1.000000,0.000000,66.932000,1910.000000,-37.911700,144.875128,2671.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
90%,4.000000,1.850000e+06,16.200000,3187.000000,4.000000,2.000000,3.000000,802.000000,240.000000,2010.000000,-37.716595,145.108900,14042.000000
95%,5.000000,2.290050e+06,20.600000,3204.000000,4.000000,3.000000,3.000000,995.000000,294.000000,2012.000000,-37.698939,145.153631,14949.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

In [24]:
df.shape

(13580, 21)

- There are 13580 rows & 20 columns in the data set.
- There seems to be some null values in 4 columns. We'll check them.
- Need to see data dictionary for more insight of columns

### Dictionary
- Rooms: Number of rooms
Price: Price in dollars
- Method:
  - S - property sold
  - SP - property sold prior
  - PI - property passed in
  - PN - sold prior not disclosed
  - SN - sold not disclosed
  - NB - no bid
  - VB - vendor bid
  - W - withdrawn prior to auction
  - SA - sold after auction
  - SS - sold after auction price not disclosed
  - N/A - price or highest bid not available.
- Type:
  - br - bedroom(s)
  - h - house,cottage,villa, semi,terrace
  - u - unit, duplex
  - t - townhouse
  - dev site - development site
  - o res - other residential.
- SellerG: Real Estate Agent
- Date: Date sold
- Distance: Distance from CBD
- Regionname: General Region (West, North West, North, North east …etc)
- Propertycount: Number of properties that exist in the suburb.
- Bedroom2 : Scraped # of Bedrooms (from different source)
- Bathroom: Number of Bathrooms
- Car: Number of carspots
- Landsize: Land Size
- BuildingArea: Building Size
- CouncilArea: Governing council for the area

## Analyzing dataset

In [27]:
have_null = 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Car'
for i in have_null:
  print(round(100*(df[i].isna().sum()/13580), 2), '% data in', i,'is null value')

47.5 % data in BuildingArea is null value
39.58 % data in YearBuilt is null value
10.08 % data in CouncilArea is null value
0.46 % data in Car is null value


In [47]:
num_cols = list(df.dtypes[df.dtypes==np.number].index)
num_cols

['Price',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Car',
 'Landsize',
 'BuildingArea',
 'YearBuilt',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [46]:
obj_cols = list(df.dtypes[df.dtypes==np.object].index)
obj_cols

['Suburb',
 'Address',
 'Type',
 'Method',
 'SellerG',
 'Date',
 'CouncilArea',
 'Regionname']

## Checking Cartegorical Variables

In [51]:
for i in obj_cols:
  print('-----', i, '------\n', df[i].value_counts(), '\n\n', sep='')

-----Suburb------
Reservoir             359
Richmond              260
Bentleigh East        249
Preston               239
Brunswick             222
                     ... 
Croydon South           1
Bullengarook            1
Bacchus Marsh           1
Beaconsfield Upper      1
Attwood                 1
Name: Suburb, Length: 314, dtype: int64


-----Address------
36 Aberfeldie St        3
1/1 Clarendon St        3
53 William St           3
13 Robinson St          3
5 Margaret St           3
                       ..
16 Reading Cl           1
203/11 Hillingdon Pl    1
3 Curtin Ct             1
20 Gordon St            1
109 North Rd            1
Name: Address, Length: 13378, dtype: int64


-----Type------
h    9449
u    3017
t    1114
Name: Type, dtype: int64


-----Method------
S     9022
SP    1703
PI    1564
VB    1199
SA      92
Name: Method, dtype: int64


-----SellerG------
Nelson                 1565
Jellis                 1316
hockingstuart          1167
Barry                  101

- We can see that address column has very high unique values. So we can't use it for analysis. We'll drop this column.